# Beer draft modelling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm_notebook
import random as random

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, precision_recall_curve, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [3]:
from my_classes import Dropper, Dummier, Vectorizer, Stemmer

In [4]:
apa_df = pd.read_csv("./data/reviews/apa_cleaned.csv")

In [5]:
apa_df.head()

,rating,look,smell,taste,feel,overall,review,date,id,name
0,3.62,4.25,3.50,3.50,3.75,3.75,From notes 3/31/13: Pours clear medium amber p...,2019-01-04,1157,Honker's Ale
1,4.06,4.00,4.25,4.00,4.00,4.00,It's funny that the best English Bitter that I...,2018-12-17,1157,Honker's Ale
2,3.93,3.75,3.75,4.00,4.00,4.00,12oz bottle dated 21SEP18 poured into my mouth...,2018-12-14,1157,Honker's Ale
3,4.35,4.00,4.00,4.75,4.00,4.25,Had this on tap when we went for dinner. I was...,2018-08-19,1157,Honker's Ale
4,3.27,3.50,3.25,3.25,3.25,3.25,As a fan of their IPA thought I would give Hon...,2018-08-18,1157,Honker's Ale


In [6]:
apa_df.describe()

,rating,look,smell,taste,feel,overall,id
count,17349.000000,17349.000000,17349.000000,17349.000000,17349.000000,17349.000000,17349.000000
mean,3.847477,3.914798,3.806233,3.848252,3.843569,3.869291,68297.891982
std,0.507472,0.500440,0.570200,0.577121,0.540289,0.541931,63931.274590
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,39.000000
25%,3.570000,3.500000,3.500000,3.500000,3.500000,3.500000,10365.000000
50%,3.900000,4.000000,4.000000,4.000000,4.000000,4.000000,59192.000000
75%,4.190000,4.250000,4.250000,4.250000,4.250000,4.250000,91566.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,315536.000000


In [8]:
#setting my binary rating at about the median
apa_df["rating_bin"] = apa_df["rating"].apply(lambda x: 1 if x > 3.9 else 0)

### Baseline

In [9]:
#checking baseline accuracy
baseline = apa_df["rating_bin"].value_counts(normalize=True).max()
baseline

0.5068303648625281

### Logistic

In [10]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs")

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.772
CV std = 0.009

Test score: 0.776


In [19]:
log_coeffs = pd.DataFrame(zip(log_pipe.steps[1][1].feature_names, log_pipe.steps[2][1].coef_[0]), columns = ["feature", "coeff"])
log_coeffs.sort_values("coeff", ascending=False).head(15)

,feature,coeff
8224,great,4.763636
5162,delicious,4.697484
6659,excellent,4.184421
6923,fantastic,3.939213
13146,perfect,3.925794
9898,juicy,3.625156
1922,beautiful,3.211588
10963,mango,3.100806
19832,wonderful,3.090032
1639,awesome,2.967771


In [21]:
for i in range(0,apa_df["review"].str.contains("dank").sum()):
    print(apa_df[apa_df["review"].str.contains("dank")].iloc[i,0])
    print(apa_df[apa_df["review"].str.contains("dank")].iloc[i,6])
    print("")

4.04
Waited to try this beer for years and then kind of forgot about it. Suddenly it appears on local shelves. It is pretty good for only having the terrible Citra hop being used (I am sorry but Psuedo Sue is terrible). I would describe this as the original juicy pale ale before the haze craze came along. It also has the benefit of having bitterness and not being overly sweet. Would I drink this regularly? Probably not as I am more in the pine, herbal, earthy, dank spectrum with hoppy beers.

4.33
Appearance: Pours a glowing, but not bright,orange/amber color like an ember from a dying campfire. Aroma: Strong citrus aroma of mandarin orange, ripe pineapple, grapefruit. A dank pine forest. Flavor: Tons of nice mandarin orange, grapefruit, ripe pinapple with a dank and resiny pine flavor. A somewhat sweeter honey (maybe toffee?) bread malt body follows. A wave of bitterness finishes on the end.  Feel: Medium body, but a heavier mouthfeel. It captures the IPA level of hop flavor bitter as

Look - a slightly opaque golden orange with a one-finger white head that sticks around and has decent lacing throughout. Smell - dank pungent hops are front and center, but you also get a nice array of hop aromas in the form of citrus (grapefruit, lemon, orange) and floral notes. There are also some sweet notes lingering in the background that might be caramel malts. Taste - very similar to the nose in that dank resin, earthy hops are very powerful, but it's balanced with the lighter side of the hop spectrum since the citrus and floral notes (like mentioned above) shine through. Again, there's a slight malty presence in the form of caramel and bread, but these notes are almost completely overpowered by the thick hop characteristics.  Feel - high carbonation, medium body, sticky mouthfeel. Overall, this is definitely an interesting example of the style. It tastes more like an imperial IPA, but it still maintains aspects of a lighter, refreshing pale ale. It's a hop bomb for sure, which 

Hazy amber orange color with a thick two finger head with good retention and leaving some laces. Superb hoppy aroma with notes of mango, lychees, fresh orange-lemon zest, clean pine, grapefruit, pineapple, flowers and bread. I am hungry, so more nose sensitive and easier to imagine this fresher. Flavor with notes of green tea, lemongrass, a bit of mango, ginger, pine, white pepper, sweet flowers, grapefruit, lemon and tangerines. Dry aftertaste with a very delimited bitterness. Light body with perfect carbonation of the tiniest bubbles given a somewhat velvet feeling. Relatively potent at 6.8% abv for and APA which is completely absent. World class APA with lots of dank and green diversity side by side zesty citrus notes. Clean and crisp as it can be. I was expecting a high bar which was delivered accordingly.

4.23
Pours a medium gold/light amber coloration with a medium, fizzy head. Smell is quite hoppy, with light biscuity malt. Notes of citrus/tropical fruit, with orange and tanger

CAN Sticky snow cream over hazy brass. 4 Orange rind, sugared grapefruit, and dank. 4 Thread of cracker and Children’s Bayer to start, slight toffee, a bit dirty, blooms into big papaya, lemondrop, grass, and flowers, then tight lemon rind bitter leave. 4.25 Scratchy linger.  Light, oily, round. 4.25 11.16 canning, 12.21 drinking. When it’s cold, it comes across thin and underhopped - APAs suffer compared to Session IPAs these days. But it thickens up as it warms, and the lean citrus opens into resin and a ton of orange - an Amarillo fest. Weedy edge to the hopping, smooth enough, crisp - this is also doing a lot of details right that other beers don’t notice, especially at just ~5%.  A shifty, faceted old-school pale ale that rewards repeated visits. 4.5

4.25
From a 16oz can: Appearance: Clear gold, good head. Smell: Citrus and some dank aroma. Taste: Nice balanced breadiness and moderate citrus bitterness. It's hoppy but you definitely get a good malt base. Overall: Picked up a four

Pours a maize color with a white near-fluffy head and lacing. This is the second in a line of five beers and I'm getting an impressive display of hop freshness so far. Tangerine, passionfruit and dank hops on the nose translates to fresh dew, biscuit or saltine cracker and a hint of lime on the tongue. The bitterness builds a bit, but at the end of the night, I think this is my favorite of Maine's offerings.

4.52
Maine Beer Co puts out so many refined, spot on beers it's almost mind blowing, and MO meets every standard they've set for their offerings. Pours a light golden brown with a skinny white head of foam. Smells clean and crisp, some citrus, slight hit of dank pine. Taste is sweet malt followed by fresh bursts of citrus fruit. Carbonation is spot on.

3.45
Bottle dated 6/4/15 so it's about a week past its drop dead date. I really like APAs, but this one doesn't float my boat. I don't get any citrus, not much floral, some dank, bitter pine but that's about it. It's nice and light

...

Poured from a 16 ounce can into a Shaker pint glass, "bottled on" date 6-1-17. APPEARANCE: pours a copper color with larger bits suspended about, two finger off-white head with excellent retention that slowly dissipated until a thin, unbroken layer of small bubbles remained, significant lacing on the glass, lighter carbonation. SMELL: bread, pineapple and a dank, earthy note. TASTE: the taste follows the nose, initially sweet pineapple and bread then an earthy, metallic note on the finish. PALATE: lighter carbonation, medium body. OVERALL: this is a good beer but it could use more carbonation, it has an assertive hop profile that could pass for an IPA, I enjoyed this beer and would have it again no problem, would recommend to anyone who enjoys the style.

3.88
16 oz. can. Hazy bright orange. Smells of sweet orange and a hefty resin aroma behind it. Cracker malts. Taste has the same bright orange up front, but it’s dank and herbal behind it. Resiny finish, though not a ton of bitterness

Pours typical Trillium orange juice.  Aroma is pineapple, mango, some piney resin and grapefruit. Flavor is less juicy than most Trillium hoppy beers, you can actually taste some pilsner malt! A bit dank and resinous with some grapefruit pith as well. Light and silky mouthfeel. Very good beer, less over the top than most other of their hop forward offerings.

4.54
The beer comes in a hazy medium gold color. The aroma kicks of huge dank Mosaic qualities, rich green herbs, orange peel, hints of nectarine and fuzzy peach. The flavor hints of apricot, grapefruit, orange zest, a doughy malt, with hints of sweetness, onto a dank and green herbaceous finish.

4.06
Not clear. Not quite as hazy as the Trillium I'm used to but still hazier than most. Two finger white head with ok retention. Lacing is also ok. Unfortunately had debris. Great smell. Tropical with a hint of dank. Nice bready malt smell also. First taste is nice with a blend of bitter pine and tropical fruit. Started with an afterta

If only all Session IPAs were this good L: Straw colored, hazy with almost a green tint. Fluffy head dissipates quickly to a thin cap. Left nice lacing. S: Citrus, lemonade, fruit cocktail, vibrant/pungent citra hops. There is a light grainy character underlying the aroma that gives way to that delightfully dank Otter Creek stank. T: Light body, citrus up front. You certainly get the citra up front. Very light malt creating a somewhat bready under tone. Delightful hop snap with spice.  F: Left my mouth puckered. It gave me that same feeling that I get after downing a lemonade on a hot day. Really refreshing, does not fatigue the pallet. Otter Creek can do no wrong in my book. This is just another example of a stellar operation.

4.51
My buddy brought this over. I didn't see the bottling date but it tasted super fresh. Poured into teku glasses. A beautiful clear golden color with slight haze and pure white ample head and decent lace. Aroma is great - citrus and tropical fruits with a bi

Hubert - Melvin  MPA - 6% abv Canned 11/17/17 (23 days ago) L - 2-finger head atop a bright yellow body. S - pine, some weediness, a little citrus, a little spicyness T - very dank. The weediness is way forward in the taste. A little citrus in there too. Some spicy rye breadiness in the middle. Only a subtle bitterness. No detectable alcohol. F - medium body and carb. Rather drying. O - I've yet to be disappointed by Melvin. This one is up there with Citradamus, but not quite at that level.

3.79
Served on tap in a pint glass Appearance – The beer is served a lightly hazed golden color with a two finger head of white foam. The head has a great level of retention, slowly fading over time to leave a good level of foamy lace on the sides of the glass. Smell – The aroma of the brew is strongest of a pine and citrus hop mixed with some lighter smells of cracker and a bit of herb. Taste - The taste begins with a rather crisp and clean cracker malt taste that has a tiny bit of tropical fruit 

It pours golden with a slight haze and about two fingers of head. The aroma is dank and piney. The taste is fairly balanced but the finish is fairly lopsided on the bitter side. Medium body and not much in terms of carbonation bite.

4.01
Got in a trade with The-Adjunct-Hippie. 16 ounce can into pint glass, canned on 7/20/2017. Pours slightly hazy/cloudy deep golden/yellow color with a 2 finger fairly dense and fluffy white head with great retention, that reduces to a large cap that lasts the whole glass. Thick foamy lacing clings down the glass, with a moderate amount of streaming carbonation retaining the cap. Fantastic appearance. Aromas of tangerine, grapefruit, peach, pineapple, passion fruit, papaya, mango, melon, berry, lemon zest, orange peel, light pepper/pine, honey, lightly toasted bread, herbal, and floral/grassy earthiness. Nice and pleasant aromas with good balance and complexity of citrus/fruity/tropical hops and moderate pale malt notes; with solid strength. Taste of bi

### Random Forest

In [22]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
vec = Vectorizer(to_vec = "review")
model = RandomForestClassifier(n_estimators=100)

#Setting up pipeline
for_pipe = Pipeline(steps = [("dropper", dropper),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
for_pipe.fit(X_train, y_train)
scores = cross_val_score(for_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(for_pipe.score(X_test, y_test)))

CV score: 0.751
CV std = 0.009

Test score: 0.757


In [23]:
for_import = pd.DataFrame(zip(for_pipe.steps[1][1].feature_names, for_pipe.steps[2][1].feature_importances_), columns = ["feature", "importance"])

In [24]:
for_import.sort_values("importance", ascending=False).head()

,feature,importance
8224,great,0.008198
5044,decent,0.007551
8589,head,0.007282
1698,bad,0.006758
3743,citrus,0.006153


### Stemming Logistic

In [26]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs")

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             ("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.769
CV std = 0.005

Test score: 0.774


In [118]:
log_coeffs = pd.DataFrame(zip(log_pipe.steps[1][1].feature_names, log_pipe.steps[2][1].coef_[0]), columns = ["feature", "coeff"])
log_coeffs.sort_values("coeff", ascending=False).head(10)

,feature,coeff
82975,nice,5.271739
56435,great,3.341485
9443,balanced,2.916788
54626,good,2.507252
104393,session,2.293725
24044,cask,2.251237
26362,citrus,2.187216
43185,excellent,2.183246
108486,smooth,2.020351
104403,session beer,1.788776


### Finding best C for logistic

In [148]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegressionCV(solver = "lbfgs", Cs=20, cv=5, max_iter=1000)

#Setting up pipeline
logcv_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
logcv_pipe.fit(X_train, y_train)

print("Train score: {0:.3f}".format(logcv_pipe.score(X_train, y_train)))

Train score: 0.864


In [153]:
logcv_pipe.steps[2][1].C_[0]

0.8286427728546842

In [248]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs", C=0.8286427728546842)

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.751
CV std = 0.014

Test score: 0.754


In [249]:
log_coeffs = pd.DataFrame(zip(log_pipe.steps[1][1].feature_names, log_pipe.steps[2][1].coef_[0]), columns = ["feature", "coeff"])
log_coeffs.sort_values("coeff", ascending=False).head(20)

,feature,coeff
5210,great,3.433925
7538,nice,3.185744
1060,balanced,2.545962
4210,excellent,2.479719
5126,good,2.304788
2966,crisp,1.691999
12471,wonderful,1.538273
9833,session,1.518206
1346,biscuity,1.506604
9354,rich,1.481384


### Trying gridsearch (no real luck)

In [191]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs", C=0.8286427728546842)

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

log_params={"vec__min_df" : [1, 0.02, 0.05],
            "vec__max_df" : [1.0, 0.9, 0.95],
            "vec__ngram_range" : [(1,1)],
            #"stem__style" : [None, "porter", "lancaster"],
            "model__C" : np.logspace(-3,0,10),
            "model__solver" : ["newton-cg", "saga", "sag"],
            "model__penalty" : ["l2"]}

log_gs=GridSearchCV(log_pipe,
                    log_params,
                    n_jobs=-1,
                    cv=3,
                    verbose=10)

#Fitting and scoring model
log_gs.fit(X_train, y_train)

scores = cross_val_score(log_gs.best_estimator_, X_train, y_train, cv=5)
print("GS score: {0:.3f}".format(log_gs.best_score_))
print("")
print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_gs.best_estimator_.score(X_test, y_test)))

Fitting 3 folds for each of 270 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   21.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:  3.9min
[Paralle

[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 266 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 269 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 271 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 273 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 274 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  7.4min
[Paralle

[Parallel(n_jobs=-1)]: Done 398 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 399 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 401 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 403 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 405 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 406 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 407 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 408 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 409 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 411 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 412 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 413 tasks      | elapsed: 11.2min
[Paralle

[Parallel(n_jobs=-1)]: Done 531 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 532 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 534 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done 535 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 537 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 538 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 539 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 541 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 542 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 543 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 544 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 545 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 546 tasks      | elapsed: 14.6min
[Paralle

[Parallel(n_jobs=-1)]: Done 664 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 665 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 666 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 667 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 668 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 669 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 670 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 671 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 673 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 674 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 675 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 676 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 677 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 678 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 679 tasks      | elapsed: 18.0min
[Paralle

[Parallel(n_jobs=-1)]: Done 797 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done 798 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done 799 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done 801 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done 802 tasks      | elapsed: 21.8min
[Parallel(n_jobs=-1)]: Done 803 tasks      | elapsed: 21.8min
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed: 22.0min finished


GS score: 0.750

CV score: 0.750
CV std = 0.016

Test score: 0.753


In [192]:
log_gs.best_estimator_

Pipeline(memory=None,
         steps=[('dropper',
                 Dropper(to_drop=['look', 'smell', 'taste', 'feel', 'overall',
                                  'date', 'id', 'name', 'rating'])),
                ('vec',
                 Vectorizer(max_df=1.0, min_df=1, ngram_range=(1, 1),
                            stop_words='english', to_vec='review')),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='newton-cg', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

### Trying SVC

In [160]:
from sklearn.svm import SVC

In [165]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = SVC(gamma="scale")

#Setting up pipeline
svc_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
svc_pipe.fit(X_train, y_train)
scores = cross_val_score(svc_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(svc_pipe.score(X_test, y_test)))

CV score: 0.748
CV std = 0.016

Test score: 0.750


### Trying a higher split

In [239]:
apa_df.describe()

,rating,look,smell,taste,feel,overall,id,rating_bin
count,4874.000000,4874.000000,4874.000000,4874.000000,4874.000000,4874.000000,4874.000000,4874.000000
mean,3.619532,3.754762,3.512302,3.612630,3.601705,3.727731,21149.727329,0.558474
std,0.540676,0.527734,0.591372,0.630672,0.606153,0.658809,29841.591009,0.496620
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,222.000000,0.000000
25%,3.320000,3.500000,3.000000,3.500000,3.250000,3.500000,2933.000000,0.000000
50%,3.680000,4.000000,3.500000,3.500000,3.500000,4.000000,10144.000000,1.000000
75%,3.980000,4.000000,4.000000,4.000000,4.000000,4.000000,27854.000000,1.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,242866.000000,1.000000


In [243]:
#setting X and y, and creating split
X=apa_df.drop(columns = ["rating_bin"]).copy()
y = X["rating"].apply(lambda x: 1 if x >= 4 else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs", C=0.8286427728546842)

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.777
CV std = 0.009

Test score: 0.802


In [246]:
X["rating"].apply(lambda x: 1 if x >= 4 else 0).value_counts(normalize=True).max()

0.7535904800984817

### Going more in depth in the best model so far

In [254]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs", C=0.8286427728546842)

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.749
CV std = 0.012

Test score: 0.743


In [257]:
lr_predict = log_pipe.predict(X_test)

print(classification_report(y_test, lr_predict))

              precision    recall  f1-score   support

           0       0.75      0.63      0.68       430
           1       0.74      0.83      0.78       545

    accuracy                           0.74       975
   macro avg       0.74      0.73      0.73       975
weighted avg       0.74      0.74      0.74       975



In [258]:
conmat = confusion_matrix(y_test, lr_predict, labels = [1,0])

pd.DataFrame(conmat,
            columns = ["p_high", "p_low"],
            index = ["high", "low"])

,p_high,p_low
high,455,90
low,161,269


In [259]:
y.mean()

0.5584735330324169

In [265]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegressionCV(solver = "lbfgs", Cs=20, cv=5, max_iter=1000, verbose=5, n_jobs=-1)

#Setting up pipeline
logcv_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
logcv_pipe.fit(X_train, y_train)

print("Train score: {0:.3f}".format(logcv_pipe.score(X_train, y_train)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 12.9min remaining: 19.3min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 15.3min finished


Train score: 0.894


In [271]:
logcv_pipe.steps[2][1].C_[0]

array([1.62377674])

In [272]:
logcv_pipe.score(X_test, y_test)

0.7405128205128205

In [273]:
#setting X and y, and creating split
X=apa_df.copy()
y = X.pop("rating_bin")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

#Setting up pipeline classes
dropper = Dropper(["look", "smell", "taste", "feel",
                    "overall", "date", "id", "name", "rating"])
stem = Stemmer(["review"], style="porter")
vec = Vectorizer(to_vec = "review")
model = LogisticRegression(solver = "lbfgs", C=1.62377674)

#Setting up pipeline
log_pipe = Pipeline(steps = [("dropper", dropper),
                             #("stem", stem),
                              ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
log_pipe.fit(X_train, y_train)
scores = cross_val_score(log_pipe, X_train, y_train, cv=5)

print("CV score: {0:.3f}".format(np.mean(scores)))
print("CV std = {0:.3f}".format(np.std(scores)))
print("")
print("Test score: {0:.3f}".format(log_pipe.score(X_test, y_test)))

CV score: 0.749
CV std = 0.013

Test score: 0.741


### Trying out the model on brewers' descriptions

In [274]:
beers = pd.read_csv("./data/beers.csv")

In [292]:
beers["review"]=beers["desc"]
clean_beers = beers.dropna()

In [301]:
#setting X and y, and creating split
X=clean_beers[["review"]].copy()
y=clean_beers["av_rating"].apply(lambda x: 1 if x > 3.6 else 0)

#Setting up pipeline
new_pipe = Pipeline(steps = [ ("vec", vec),
                              ("model", model)])

#Fitting and scoring model
y_pred = new_pipe.predict(X)

In [303]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.14      0.07      0.10        14
           1       0.71      0.84      0.77        38

    accuracy                           0.63        52
   macro avg       0.43      0.46      0.43        52
weighted avg       0.56      0.63      0.59        52



In [304]:
conmat = confusion_matrix(y, y_pred, labels = [1,0])

pd.DataFrame(conmat,
            columns = ["p_high", "p_low"],
            index = ["high", "low"])

,p_high,p_low
high,32,6
low,13,1


In [306]:
clean_beers["rating_pred"] = y_pred

C:\Users\James\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
for i in range(0,7):
    print(clean_beers[clean_beers["rating_pred"] == 0].iloc[i,10])
    print("")

This 4.2% Kentish ale (bottle 4.5%) was first brewed in 1990 to celebrate the Battle of Britain which was fought in the skies above Kent 50 years earlier. The beer is named after the legendary Spitfire aeroplane designed by RJ Mitchell. The versatility of the aircraft and the courage of its pilots were essential to victory and were a key symbol of the spirit of that time.

The pint of pints. Belhaven Best is the main man of draught ales. Best colour, best flavour, best balance - the pint for all occasions. We Scots are often not known for blowing our own trumpets, modest souls that we are, but we've named this Best for a reason. We're sure you'll agree.

5.5% ABV 31 IBU'S

A general term for what is perhaps the highest expression of the brewer's craft. Amber not filtered and true to style. A natural with fish and chips.

Cwrw Braf is a clean-drinking, amber-coloured ale with a light bitterness and gentle hop aroma. A session bitter yet, with all the flavours and quality you would expec